In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
ticker_symbol='TCS.NS'
ticket=yf.Ticker(ticker_symbol)
data=ticket.history(period="1y", start="2023-01-01", end="2023-12-31", interval="1d")
import warnings
warnings.filterwarnings("ignore")
# Simple Moving Average (SMA)
def mean(data, i, period): # i=rows period = no of day SMA
  if i < period - 1:
    return None
  else:
    return data['Close'].iloc[i - period + 1:i + 1].mean()

data['SMA_30']=None
for i in range(0,len(data)):
  data['SMA_30'].iloc[i]=mean(data, i, 30)


# EXPONENTIAL MOVING AVERAGE
def ema(day):
  s = 2/(1+day)
  for i in range (0,len(data)):
    if i==0:
      data['EMA'].iloc[i]=data['Close'].iloc[i]
    else:
      data['EMA'].iloc[i]=s*(data['Close'].iloc[i])+(1-s)*data['EMA'].iloc[i-1]
  #return data
data['EMA']=None
ema(12)

#Part 1
#Bollinger band
def mean(data, i, period):
  if i < period - 1:
    return None
  else:
    return data['Close'].iloc[i - period + 1:i + 1].mean()

data['SMA_30']=None
for i in range(0,len(data)):
  data['SMA_30'].iloc[i]=mean(data, i, 30)
def moving_std(data, i, period):
  if i < period - 1:
    return None
  else:
    return data['Close'].iloc[i - period + 1:i + 1].std()

data['MSD_30']=None
for i in range(0,len(data)):
  data['MSD_30'].iloc[i]=moving_std(data, i, 30)
     #return data
def bollinger_band(data,period):
  data['Middle_Band']=data['SMA_30']
  data['Upper_Band']=data['SMA_30']+(2*data['MSD_30'])
  data['Lower_Band']=data['SMA_30']-(2*data['MSD_30'])
  return data


bollinger_band(data,30)
import matplotlib.pyplot as plt
def ema(day):
  s = 2/(1+day)
  data['EMA']=None
  for i in range (0,len(data)):
    if i==0:
      data['EMA'].iloc[i]=data['Close'].iloc[i]
    else:
      data['EMA'].iloc[i]=s*(data['Close'].iloc[i])+(1-s)*data['EMA'].iloc[i-1]
  return data['EMA']
def MACD(data,period1,period2,period3):
  data['EMA_12']=ema(period1)
  data['EMA_26']=ema(period2)
  data['MACD']=None
  for i in range(len(data)):
        if data['EMA_12'][i] is not None and data['EMA_26'][i] is not None:
           data['MACD'].iloc[i] = data['EMA_12'].iloc[i] - data['EMA_26'].iloc[i]
        else:
            data['MACD'].iloc[i] = None
  data['Signal']=None
  x=2/(1+period3)
  for i in range (0,len(data)):
    if i==0:
      data['Signal'].iloc[i]=data['MACD'].iloc[i]
    else:
      data['Signal'].iloc[i]=x*data['MACD'].iloc[i]+(1-x)*data['Signal'].iloc[i-1]
MACD(data,12,26,9)

#Stochastic Oscillator(SO)
def SO(data,period):
  data['Lowest_low']=data['Low'].rolling(window=period).min()
  data['Maximum_high']=data['High'].rolling(window=period).max()
  data['stochastic oscillator']=(data['Close']-data['Lowest_low'])/(data['Maximum_high']-data['Lowest_low'])*100
  return data

SO(data,14)

#Relative Strength Index
def rsi(data,length):
  data['delta']=data['Close'].diff()
  data['gain']=data['delta'].where(data['delta']>0,0)
  data['loss']=data['delta'].where(data['delta']<0,0)
  data['avg_gain']=data['gain'].rolling(window=length).mean()
  data['avg_loss']=data['loss'].rolling(window=length).mean()
  data['rs']=data['avg_gain']/data['avg_loss']
  data['rsi']=100-(100/(1+data['rs']))
  return data
rsi(data,14)

#Average True Range
def atr(data,period):
  data['H-L']=data['High']-data['Low']
  data['H-Cp']=data['High']-data['Close'].shift(1)
  data['L-Cp']=data['Low']-data['Close'].shift(1)
  data['TR']=data[['H-L','H-Cp','L-Cp']].max(axis=1)
  data['ATR']=data['TR'].rolling(window=period).mean()
  return data
atr(data,14)

#Average Directional Index
#Average Directional Index
def dir_m(data):
    data['+DM'] = data['High']-data['High'].shift(1)
    data['+DM'] = data['+DM'].apply(lambda x: x if x > 0 else 0)
    data['-DM'] = data['Low'].shift(1) - data['Low']
    data['+DM'] = data['+DM'].apply(lambda x: x if x > 0 else 0)
def emp(data, period):
    s = 2 / (1 + period)
    data['EMP'] = 0
    for i in range(1, len(data)):
        data['EMP'].iloc[i] = s * (data['+DM'].iloc[i]) + (1 - s) * data['EMP'].iloc[i - 1]

def emn(data, period):
    s = 2 / (1 + period)
    data['EMN'] = 0
    for i in range(1, len(data)):
        data['EMN'].iloc[i] = s * (data['-DM'].iloc[i]) + (1 - s) * data['EMN'].iloc[i - 1]

def dir_m_index(data):
    data['+DI'] = 100 * data['EMP'] / data['ATR']
    data['-DI'] = 100 * data['EMN'] / data['ATR']
    data['+DI'] = data['+DI'].fillna(0)
    data['-DI'] = data['-DI'].fillna(0)
    data['DX'] = (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI'])) * 100
    data['ADX'] = data['DX'].rolling(window=14).mean()
    data['ADX'] = data['ADX'].fillna(0)

dir_m(data)
emp(data, 14)
emn(data, 14)
dir_m_index(data)

print(data[['Close', '+DM', '-DM', '+DI', '-DI','ADX']])



                                 Close        +DM        -DM        +DI  \
Date                                                                      
2023-01-02 00:00:00+05:30  3104.663818   0.000000        NaN   0.000000   
2023-01-03 00:00:00+05:30  3152.165283  50.594997  -9.614702   0.000000   
2023-01-04 00:00:00+05:30  3155.306641   6.996943 -38.933924   0.000000   
2023-01-05 00:00:00+05:30  3151.927002   9.471147   4.046170   0.000000   
2023-01-06 00:00:00+05:30  3057.163086   0.000000  78.009902   0.000000   
...                                ...        ...        ...        ...   
2023-12-22 00:00:00+05:30  3760.965088  38.603107 -18.342580  42.127756   
2023-12-26 00:00:00+05:30  3732.984131   0.000000 -27.685903  36.239151   
2023-12-27 00:00:00+05:30  3748.375977   0.000000  21.784865  32.453761   
2023-12-28 00:00:00+05:30  3737.262207  19.473673 -23.702839  31.215091   
2023-12-29 00:00:00+05:30  3730.869385   0.000000  26.260037  26.716900   

                        

In [2]:
#Signal for bollimger band
def signal_bollinger_bands(data):
  data['Signal_Bollinger']=0
  data.loc[data['Close'] > data['Upper_Band'], 'Signal_Bollinger'] = -1  # Sell signal
  data.loc[data['Close'] < data['Lower_Band'], 'Signal_Bollinger'] = 1 # Buy signal
  buy_signals = len(data[data['Signal_Bollinger'] == 1])
  sell_signals = len(data[data['Signal_Bollinger'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")  # Buy signal
signal_bollinger_bands(data)
print(data[['Signal_Bollinger','Close']].tail(20))



Number of Buy signals: 10
Number of Sell signals: 13
                           Signal_Bollinger        Close
Date                                                    
2023-12-01 00:00:00+05:30                 0  3453.763916
2023-12-04 00:00:00+05:30                 0  3454.550293
2023-12-05 00:00:00+05:30                 0  3473.385010
2023-12-06 00:00:00+05:30                -1  3544.689941
2023-12-07 00:00:00+05:30                -1  3555.311523
2023-12-08 00:00:00+05:30                 0  3566.917236
2023-12-11 00:00:00+05:30                 0  3582.850342
2023-12-12 00:00:00+05:30                 0  3611.569092
2023-12-13 00:00:00+05:30                 0  3534.313721
2023-12-14 00:00:00+05:30                 0  3606.799072
2023-12-15 00:00:00+05:30                -1  3797.355225
2023-12-18 00:00:00+05:30                -1  3795.584717
2023-12-19 00:00:00+05:30                 0  3753.293701
2023-12-20 00:00:00+05:30                 0  3717.739502
2023-12-21 00:00:00+05:30          

In [3]:
def signal_MACD(data):
  data['Signal_MACD']=0
  data.loc[data['MACD'] > data['Signal'], 'Signal_MACD'] = 1
  data.loc[data['MACD'] < data['Signal'], 'Signal_MACD'] = -1
  buy_signals = len(data[data['Signal_MACD'] == 1])
  sell_signals = len(data[data['Signal_MACD'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data
signal_MACD(data)
print(data[['Signal_MACD','Close']].tail(20))

Number of Buy signals: 139
Number of Sell signals: 105
                           Signal_MACD        Close
Date                                               
2023-12-01 00:00:00+05:30            1  3453.763916
2023-12-04 00:00:00+05:30            1  3454.550293
2023-12-05 00:00:00+05:30            1  3473.385010
2023-12-06 00:00:00+05:30            1  3544.689941
2023-12-07 00:00:00+05:30            1  3555.311523
2023-12-08 00:00:00+05:30            1  3566.917236
2023-12-11 00:00:00+05:30            1  3582.850342
2023-12-12 00:00:00+05:30            1  3611.569092
2023-12-13 00:00:00+05:30            1  3534.313721
2023-12-14 00:00:00+05:30            1  3606.799072
2023-12-15 00:00:00+05:30            1  3797.355225
2023-12-18 00:00:00+05:30            1  3795.584717
2023-12-19 00:00:00+05:30            1  3753.293701
2023-12-20 00:00:00+05:30            1  3717.739502
2023-12-21 00:00:00+05:30            1  3725.066650
2023-12-22 00:00:00+05:30            1  3760.965088
2023-12-2

In [4]:
def signal_Stochastic_Oscillator(data):
  data['Signal_SO']=0
  data.loc[data['stochastic oscillator'] > 80, 'Signal_SO'] = -1
  data.loc[data['stochastic oscillator'] < 20, 'Signal_SO'] = 1
  buy_signals = len(data[data['Signal_SO'] == 1])
  sell_signals = len(data[data['Signal_SO'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data
signal_Stochastic_Oscillator(data)
print(data[['Signal_SO','Close']].tail(20))

Number of Buy signals: 48
Number of Sell signals: 72
                           Signal_SO        Close
Date                                             
2023-12-01 00:00:00+05:30         -1  3453.763916
2023-12-04 00:00:00+05:30         -1  3454.550293
2023-12-05 00:00:00+05:30         -1  3473.385010
2023-12-06 00:00:00+05:30         -1  3544.689941
2023-12-07 00:00:00+05:30         -1  3555.311523
2023-12-08 00:00:00+05:30         -1  3566.917236
2023-12-11 00:00:00+05:30         -1  3582.850342
2023-12-12 00:00:00+05:30         -1  3611.569092
2023-12-13 00:00:00+05:30          0  3534.313721
2023-12-14 00:00:00+05:30         -1  3606.799072
2023-12-15 00:00:00+05:30         -1  3797.355225
2023-12-18 00:00:00+05:30         -1  3795.584717
2023-12-19 00:00:00+05:30          0  3753.293701
2023-12-20 00:00:00+05:30          0  3717.739502
2023-12-21 00:00:00+05:30          0  3725.066650
2023-12-22 00:00:00+05:30          0  3760.965088
2023-12-26 00:00:00+05:30          0  3732.9841

In [5]:
def signal_atr(data):
  multiplier_atr=1.4
  data['Buy'] = data['Close'].shift(1) + (multiplier_atr* data['ATR'])
  data['Sell'] = data['Close'].shift(1) - (multiplier_atr* data['ATR'])
  data['Signal_ATR']=0
  data.loc[data['Close'] > data['Buy'], 'Signal_ATR'] = 1
  data.loc[data['Close'] < data['Sell'], 'Signal_ATR'] = -1
  buy_signals = len(data[data['Signal_ATR'] == 1])
  sell_signals = len(data[data['Signal_ATR'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data['Signal_ATR']
signal_atr(data)
print(data[['Signal_ATR','Close']].tail(20))

Number of Buy signals: 9
Number of Sell signals: 4
                           Signal_ATR        Close
Date                                              
2023-12-01 00:00:00+05:30           0  3453.763916
2023-12-04 00:00:00+05:30           0  3454.550293
2023-12-05 00:00:00+05:30           0  3473.385010
2023-12-06 00:00:00+05:30           1  3544.689941
2023-12-07 00:00:00+05:30           0  3555.311523
2023-12-08 00:00:00+05:30           0  3566.917236
2023-12-11 00:00:00+05:30           0  3582.850342
2023-12-12 00:00:00+05:30           0  3611.569092
2023-12-13 00:00:00+05:30          -1  3534.313721
2023-12-14 00:00:00+05:30           0  3606.799072
2023-12-15 00:00:00+05:30           1  3797.355225
2023-12-18 00:00:00+05:30           0  3795.584717
2023-12-19 00:00:00+05:30           0  3753.293701
2023-12-20 00:00:00+05:30           0  3717.739502
2023-12-21 00:00:00+05:30           0  3725.066650
2023-12-22 00:00:00+05:30           0  3760.965088
2023-12-26 00:00:00+05:30      

In [6]:
def avg_dir_index(data):
  minimum_adx=25
  data['Signal_DI']=0
  data.loc[(data['+DI'] > data['-DI']) & (data['ADX']>minimum_adx) , 'Signal_DI'] = 1
  data.loc[(data['+DI'] < data['-DI']) & (data['ADX']>minimum_adx) , 'Signal_DI'] = -1
  buy_signals = len(data[data['Signal_DI'] == 1])
  sell_signals = len(data[data['Signal_DI'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data['Signal_DI']
avg_dir_index(data)
print(data[['Signal_DI','Close']].tail(20))



Number of Buy signals: 179
Number of Sell signals: 39
                           Signal_DI        Close
Date                                             
2023-12-01 00:00:00+05:30          1  3453.763916
2023-12-04 00:00:00+05:30          1  3454.550293
2023-12-05 00:00:00+05:30          1  3473.385010
2023-12-06 00:00:00+05:30          1  3544.689941
2023-12-07 00:00:00+05:30          1  3555.311523
2023-12-08 00:00:00+05:30          1  3566.917236
2023-12-11 00:00:00+05:30          1  3582.850342
2023-12-12 00:00:00+05:30          1  3611.569092
2023-12-13 00:00:00+05:30          1  3534.313721
2023-12-14 00:00:00+05:30          1  3606.799072
2023-12-15 00:00:00+05:30          1  3797.355225
2023-12-18 00:00:00+05:30          1  3795.584717
2023-12-19 00:00:00+05:30          1  3753.293701
2023-12-20 00:00:00+05:30          1  3717.739502
2023-12-21 00:00:00+05:30          1  3725.066650
2023-12-22 00:00:00+05:30          1  3760.965088
2023-12-26 00:00:00+05:30          1  3732.984

In [7]:
#RSI
def rsi(data):
  data['Signal_rsi']=0
  data.loc[data['rsi'] < 30 , 'Signal_rsi'] = 1
  data.loc[data['rsi'] > 70 , 'Signal_rsi'] = -1
  buy_signals = len(data[data['Signal_rsi'] == 1])
  sell_signals = len(data[data['Signal_rsi'] == -1])
  print(f"Number of Buy signals: {buy_signals}")
  print(f"Number of Sell signals: {sell_signals}")
  return data['Signal_rsi']
rsi(data)
print(data[['Signal_rsi','Close']].tail(20))

Number of Buy signals: 89
Number of Sell signals: 143
                           Signal_rsi        Close
Date                                              
2023-12-01 00:00:00+05:30          -1  3453.763916
2023-12-04 00:00:00+05:30          -1  3454.550293
2023-12-05 00:00:00+05:30          -1  3473.385010
2023-12-06 00:00:00+05:30          -1  3544.689941
2023-12-07 00:00:00+05:30          -1  3555.311523
2023-12-08 00:00:00+05:30          -1  3566.917236
2023-12-11 00:00:00+05:30          -1  3582.850342
2023-12-12 00:00:00+05:30          -1  3611.569092
2023-12-13 00:00:00+05:30          -1  3534.313721
2023-12-14 00:00:00+05:30          -1  3606.799072
2023-12-15 00:00:00+05:30          -1  3797.355225
2023-12-18 00:00:00+05:30          -1  3795.584717
2023-12-19 00:00:00+05:30          -1  3753.293701
2023-12-20 00:00:00+05:30          -1  3717.739502
2023-12-21 00:00:00+05:30          -1  3725.066650
2023-12-22 00:00:00+05:30          -1  3760.965088
2023-12-26 00:00:00+05:30   